#  소프트맥스 회귀로 MNIST 데이터 분류하기

* 총 60,000개의 훈련 데이터와 레이블, 총 10,000개의 테스트 데이터와 레이블로 구성
* 레이블은 0부터 9까지 총 10개
* 28 픽셀 × 28 픽셀 = 784 픽셀이므로, 각 이미지를 총 784의 원소를 가진 벡터


In [ ]:
'''
for X, Y in data_loader:
    # 입력 이미지를 [batch_size × 784]의 크기로 reshape
    # 레이블은 원-핫 인코딩
  X = X.view(-1, 28*28)
'''

## 2. 토치비전(torchvision) 소개하기
* torchvision은 유명한 데이터셋들, 이미 구현되어져 있는 유명한 모델들, 일반적인 이미지 전처리 도구들을 포함하고 있는 패키지가 있다
* 자연어 처리를 위해서는 토치텍스트(torchtext)라는 패키지가 있다

## 3. 분류기 구현을 위한 사전 설정

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)


# 랜덤 시드 고정
# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


# hyperparameters
training_epochs = 15
batch_size = 100

다음 기기로 학습합니다: cuda


## 4. MNIST 분류기 구현하기

In [ ]:
# MNIST dataset
# torchvision.datasets.dsets.MNIST
mnist_train = dsets.MNIST(root='A:/chchdata/data/MNIST_data/', #MNIST 데이터를 다운로드 받을 경로
                          train=True, # 훈련 데이터를 받음, False면 테스트 데이터를 받음
                          transform=transforms.ToTensor(),# 파이토치 텐서로 변환
                          download=True) #해당 경로에 MNIST 데이터가 없다면 다운로드 받겠다는 의미

mnist_test = dsets.MNIST(root='A:/chchdata/data/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



9913344it [00:18, 528822.83it/s]                             


Extracting A:/chchdata/data/MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to A:/chchdata/data/MNIST_data/MNIST\raw



29696it [00:00, 169078.05it/s]                          


Extracting A:/chchdata/data/MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to A:/chchdata/data/MNIST_data/MNIST\raw



1649664it [00:41, 39458.93it/s]                             


Extracting A:/chchdata/data/MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to A:/chchdata/data/MNIST_data/MNIST\raw



5120it [00:00, ?it/s]                   

Extracting A:/chchdata/data/MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to A:/chchdata/data/MNIST_data/MNIST\raw




C:\Users\sswwd\anaconda3\envs\chch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                                          batch_size=batch_size, # 배치 크기는 100
                                          shuffle=True,
                                          drop_last=True) #drop_last는 마지막 배치를 버릴 것인지를 의미

NameError: name 'batch_size' is not defined

drop_last를 하는 이유를 이해하기 위해서 1,000개의 데이터가 있다고 했을 때, 배치 크기가 128이라고 해봅시다. 1,000을 128로 나누면 총 7개가 나오고 나머지로 104개가 남습니다. 이때 104개를 마지막 배치로 한다고 하였을 때 128개를 충족하지 못하였으므로 104개를 그냥 버릴 수도 있습니다. 이때 마지막 배치를 버리려면 drop_last=True를 해주면 됩니다. 이는 **다른 미니 배치보다 개수가 적은 마지막 배치를 경사 하강법에 사용하여 마지막 배치가 상대적으로 과대 평가되는 현상을 막아줍니다.**